In [2]:
# 1
# Chance Wiese

In [3]:
pip install pymssql

Note: you may need to restart the kernel to use updated packages.


In [4]:
# 2
import pandas as pd
import pymssql

conn = pymssql.connect(
     host='stairway.usu.edu', # Server name goes in quotes.
     user='dunn2100', # Username goes in quotes.
     password='databases4ever', # Password goes in quotes
     database='overland') # Database to use goes in quotes

query = """
With MemberCount AS (
    SELECT HHID, COUNT(*) AS Members
    FROM Guest
    GROUP BY HHID
),
SeasonPassCount AS (
    SELECT HHID, COUNT(*) AS SPCount
    FROM Guest
    WHERE SeasonPass = 'y'
    GROUP BY HHID
),
Under18Count AS (
    SELECT HHID, COUNT(*) AS U18Count
    FROM Guest
    WHERE Age < 18
    GROUP BY HHID
),
VisitCount AS (
    SELECT HHID, COUNT(DISTINCT VisID) AS VisitCt
    FROM Guest g
    JOIN Visit v ON g.GuestID = v.CustID
    GROUP BY g.HHID
)
SELECT h.HHID, Region, HHI AS Income, Lname AS Lname, m.Members, sp.SPCount, u.U18Count, v.VisitCt
FROM Household h
JOIN Guest g ON h.HHID = g.HHID
JOIN MemberCount m ON h.HHID = m.HHID
LEFT JOIN SeasonPassCount sp ON h.HHID = sp.HHID
LEFT JOIN Under18Count u ON h.HHID = u.HHID
LEFT JOIN VisitCount v ON h.HHID = v.HHID
GROUP BY h.HHID, Region, HHI, Lname, m.Members, sp.SPCount, u.U18Count, v.VisitCt
ORDER BY h.HHID;
"""

df = pd.read_sql(query, conn, index_col='HHID')
print(df)

          Region  Income        Lname  Members  SPCount  U18Count  VisitCt
HHID                                                                      
1     Western PA  153000       Willis        2      NaN       1.0     12.0
2     Western PA  129000      Simpson        3      NaN       2.0      6.0
3          Other   68000  Sonic-Smith        1      NaN       NaN      6.0
4     Central PA   98000      Elliott        1      NaN       NaN      3.0
5     Central PA   52000        Owens        1      1.0       NaN      6.0
...          ...     ...          ...      ...      ...       ...      ...
7151  Central PA   36000     Mitchell        1      NaN       NaN      2.0
7152  Western PA   85000     Mitchell        1      NaN       NaN      6.0
7153     DC Area   72000        Green        1      NaN       NaN      5.0
7154  Central PA   48000    Schroeder        1      NaN       NaN      3.0
7155  Western PA  111000         Carr        1      NaN       1.0      3.0

[7155 rows x 7 columns]


C:\Users\Chanc\AppData\Local\Temp\ipykernel_12996\4280545926.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, index_col='HHID')


In [5]:
# 3
import numpy as np
df['SPCount'] = df['SPCount'].replace(np.nan, 0)
df['U18Count'] = df['U18Count'].replace(np.nan, 0)
df['VisitCt'] = df['VisitCt'].replace(np.nan, 0)
print(df)

          Region  Income        Lname  Members  SPCount  U18Count  VisitCt
HHID                                                                      
1     Western PA  153000       Willis        2      0.0       1.0     12.0
2     Western PA  129000      Simpson        3      0.0       2.0      6.0
3          Other   68000  Sonic-Smith        1      0.0       0.0      6.0
4     Central PA   98000      Elliott        1      0.0       0.0      3.0
5     Central PA   52000        Owens        1      1.0       0.0      6.0
...          ...     ...          ...      ...      ...       ...      ...
7151  Central PA   36000     Mitchell        1      0.0       0.0      2.0
7152  Western PA   85000     Mitchell        1      0.0       0.0      6.0
7153     DC Area   72000        Green        1      0.0       0.0      5.0
7154  Central PA   48000    Schroeder        1      0.0       0.0      3.0
7155  Western PA  111000         Carr        1      0.0       1.0      3.0

[7155 rows x 7 columns]


In [6]:
# 4
df['VisPP'] = df['VisitCt'] / df['Members']
print(df[['Lname', 'Members', 'VisitCt', 'VisPP']])

            Lname  Members  VisitCt  VisPP
HHID                                      
1          Willis        2     12.0    6.0
2         Simpson        3      6.0    2.0
3     Sonic-Smith        1      6.0    6.0
4         Elliott        1      3.0    3.0
5           Owens        1      6.0    6.0
...           ...      ...      ...    ...
7151     Mitchell        1      2.0    2.0
7152     Mitchell        1      6.0    6.0
7153        Green        1      5.0    5.0
7154    Schroeder        1      3.0    3.0
7155         Carr        1      3.0    3.0

[7155 rows x 4 columns]


In [7]:
# 5
df['IsLocal'] = np.where(
    df['Region'] == 'Central PA',
    1, 0)
print(df[['Lname', 'Region', 'IsLocal']])

            Lname      Region  IsLocal
HHID                                  
1          Willis  Western PA        0
2         Simpson  Western PA        0
3     Sonic-Smith       Other        0
4         Elliott  Central PA        1
5           Owens  Central PA        1
...           ...         ...      ...
7151     Mitchell  Central PA        1
7152     Mitchell  Western PA        0
7153        Green     DC Area        0
7154    Schroeder  Central PA        1
7155         Carr  Western PA        0

[7155 rows x 3 columns]


In [8]:
# 6
dflocal = df[df['IsLocal'] == 1]
print(dflocal)

          Region  Income      Lname  Members  SPCount  U18Count  VisitCt  \
HHID                                                                       
4     Central PA   98000    Elliott        1      0.0       0.0      3.0   
5     Central PA   52000      Owens        1      1.0       0.0      6.0   
6     Central PA  159000  Underwood        2      2.0       1.0     18.0   
7     Central PA  138000       Wise        1      0.0       1.0      5.0   
8     Central PA   78000      Tyler        6      0.0       4.0     22.0   
...          ...     ...        ...      ...      ...       ...      ...   
7143  Central PA   80000     Molina        1      0.0       0.0      9.0   
7145  Central PA  134000      Scott        5      0.0       4.0     33.0   
7149  Central PA  139000     Watson        2      0.0       0.0     71.0   
7151  Central PA   36000   Mitchell        1      0.0       0.0      2.0   
7154  Central PA   48000  Schroeder        1      0.0       0.0      3.0   

          V

In [9]:
# 7
dflocal.drop(columns=['IsLocal', 'Region'], inplace=True)
print(dflocal)

      Income      Lname  Members  SPCount  U18Count  VisitCt      VisPP
HHID                                                                   
4      98000    Elliott        1      0.0       0.0      3.0   3.000000
5      52000      Owens        1      1.0       0.0      6.0   6.000000
6     159000  Underwood        2      2.0       1.0     18.0   9.000000
7     138000       Wise        1      0.0       1.0      5.0   5.000000
8      78000      Tyler        6      0.0       4.0     22.0   3.666667
...      ...        ...      ...      ...       ...      ...        ...
7143   80000     Molina        1      0.0       0.0      9.0   9.000000
7145  134000      Scott        5      0.0       4.0     33.0   6.600000
7149  139000     Watson        2      0.0       0.0     71.0  35.500000
7151   36000   Mitchell        1      0.0       0.0      2.0   2.000000
7154   48000  Schroeder        1      0.0       0.0      3.0   3.000000

[2877 rows x 7 columns]


C:\Users\Chanc\AppData\Local\Temp\ipykernel_12996\3600868607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dflocal.drop(columns=['IsLocal', 'Region'], inplace=True)


In [10]:
# 8
dflocal.dropna(inplace=True)

conditions = [
    dflocal['Income'] < 50000,
    dflocal['Income'] < 100000,
    dflocal['Income'] < 150000,
    True
]
values = ['Under $50K', '$50-100K', '$100-150K', 'Over $150K']
dflocal['IncomeCat'] = np.select(conditions, values)
print(dflocal[['Lname', 'Income', 'IncomeCat', 'VisPP']])

          Lname  Income   IncomeCat      VisPP
HHID                                          
4       Elliott   98000    $50-100K   3.000000
5         Owens   52000    $50-100K   6.000000
6     Underwood  159000  Over $150K   9.000000
7          Wise  138000   $100-150K   5.000000
8         Tyler   78000    $50-100K   3.666667
...         ...     ...         ...        ...
7143     Molina   80000    $50-100K   9.000000
7145      Scott  134000   $100-150K   6.600000
7149     Watson  139000   $100-150K  35.500000
7151   Mitchell   36000  Under $50K   2.000000
7154  Schroeder   48000  Under $50K   3.000000

[2877 rows x 4 columns]


C:\Users\Chanc\AppData\Local\Temp\ipykernel_12996\4115604726.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dflocal.dropna(inplace=True)
C:\Users\Chanc\AppData\Local\Temp\ipykernel_12996\4115604726.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dflocal['IncomeCat'] = np.select(conditions, values)


In [50]:
# 9
analysis = dflocal[['VisPP', 'IncomeCat']].groupby('IncomeCat').agg(['mean', 'median', 'count'])
print(analysis, 
"""\n
Yes, there is a relationship between income and number of visits per person among local customers. 
The analysis shows that average VisPP consistently increases with income, from 3.89 for 'Under $50K' to 7.53 for 'Over $150K'.
This shows a positive correlation between income and visit frequency.
""")

               VisPP                
                mean    median count
IncomeCat                           
$100-150K   6.030133  5.333333  1122
$50-100K    4.806644  4.000000  1250
Over $150K  7.532885  7.000000   446
Under $50K  3.891525  4.000000    59 

Yes, there is a relationship between income and number of visits per person among local customers. 
The analysis shows that average VisPP consistently increases with income, from 3.89 for 'Under $50K' to 7.53 for 'Over $150K'.
This shows a positive correlation between income and visit frequency.



In [76]:
# 10
df_corr = dflocal[['Income', 'Members', 'SPCount', 'U18Count', 'VisPP']].corr()
print(df_corr['VisPP'],
"""\n
The variable with the strongest positive relationship to visits per person (VisPP) is Income (0.286).
The variable with the weakest relationship is Members (-0.118).
This shows that higher income is associated with more visits, while number of members has very little impact on visit frequency.
"""
)

Income      0.285967
Members    -0.118143
SPCount     0.233857
U18Count   -0.119280
VisPP       1.000000
Name: VisPP, dtype: float64 

The variable with the strongest positive relationship to visits per person (VisPP) is Income (0.286).
The variable with the weakest relationship is Members (-0.118).
This shows that higher income is associated with more visits, while number of members has very little impact on visit frequency.

